# Miner Notebook

This notebook is used for experimentation during the implementation of the Zimcoin miner component.

In [30]:
import sys
import logging
import numpy as np
import miner
from miner import ZimcoinMiner
from Library.opencl_information import opencl_information
from blocks import Block, mine_block
from transactions import Transaction

from importlib import reload

## Configure Logging

In [31]:
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S',
        handlers=[
        logging.FileHandler("miner_notebook.log"),
        logging.StreamHandler(sys.stdout)
    ])

## Show the available OpenCL Platforms

In [32]:
info = opencl_information()
info.print_full_info()


OpenCL Platforms and Devices
Platform 0 - Name: NVIDIA CUDA
Platform 0 - Vendor: NVIDIA Corporation
Platform 0 - Version: OpenCL 3.0 CUDA 11.7.89
Platform 0 - Profile: FULL_PROFILE
 --------------------------------------------------------
 Device - Name: NVIDIA GeForce RTX 3060
 Device - Type: ALL | GPU
 Device - Max Clock Speed: 1867 Mhz
 Device - Compute Units: 28
 Device - Local Memory: 48 KB
 Device - Constant Memory: 64 KB
 Device - Global Memory: 12 GB
 Device - Max Buffer/Image Size: 3011 MB
 Device - Max Work Group Size: 1024


Platform 1 - Name: Portable Computing Language
Platform 1 - Vendor: The pocl project
Platform 1 - Version: OpenCL 1.2 pocl 1.6, None+Asserts, LLVM 9.0.1, RELOC, SLEEF, DISTRO, POCL_DEBUG
Platform 1 - Profile: FULL_PROFILE
 --------------------------------------------------------
 Device - Name: pthread-AMD Ryzen 5 3600 6-Core Processor
 Device - Type: ALL | CPU
 Device - Max Clock Speed: 4208 Mhz
 Device - Compute Units: 12
 Device - Local Memory: 512 K

## Create the Miner

In [33]:
reload(miner)

miner = ZimcoinMiner(0, 0)
print('Threads:', miner.thread_count)

Threads: 28672


## Calculate Hash Test

The input data does not fit neatly into 32 byte blocks so some padding and un-padding will be needed.

### Bit Shifting Tests

In [85]:
nonce = np.ulonglong(np.iinfo(np.ulonglong).max  // 2)
#nonce = np.ulonglong(5)

print('nonce:', nonce)
print('Nonce Bytes:', nonce.nbytes)
print('Nonce to Bytes :', nonce.tobytes())

print('64 bit integers :', np.frombuffer(nonce.tobytes(), dtype=np.uint64))
print('32 bit integers :', np.frombuffer(nonce.tobytes(), dtype=np.uint32))


int_nonce = int(nonce)
print('Integer Nonce:', int_nonce)
print(int_nonce & 0xFFFFFFFF)
print(int_nonce >> 32)

nonce: 5
Nonce Bytes: 8
Nonce to Bytes : b'\x05\x00\x00\x00\x00\x00\x00\x00'
64 bit integers : [5]
32 bit integers : [5 0]
Integer Nonce: 5
5
0


### Helper Functions to create test blocks 

In [41]:
def get_test_first_block() -> Block:
    """
    Get a block representing the first in a chain for testing.
    """
    return Block(
            bytes.fromhex('0000000000000000000000000000000000000000000000000000000000000000'),
            0,
            bytes.fromhex('4f3ea27a7af06cbe53911d4fb9326730d435255a'),
            [],
            1626626569,
            100000,
            bytes.fromhex('0000193f7397d8ed1a4991d91f8b8d2e55eb56915e884d435de7bbf0b183f335'),
            55419)

def get_transactions_block() -> Block:
    """
    Get a block with transactions for testing.
    """
    return Block(
            bytes.fromhex('0000193f7397d8ed1a4991d91f8b8d2e55eb56915e884d435de7bbf0b183f335'),
            1,
            bytes.fromhex('4f3ea27a7af06cbe53911d4fb9326730d435255a'),
            [
                Transaction(
                    bytes.fromhex("9e09208d54c012c0844cf17cfbb175157516dc90"),
                    bytes.fromhex("4f3ea27a7af06cbe53911d4fb9326730d435255a"),
                    bytes.fromhex("3056301006072a8648ce3d020106052b8104000a034200041a719dc420fdbdeef447e90a6368b9486d4afbacd900f6d9d5f62692dfa9ecb695999af4fcf61bdc523021b3aef2b84344b7c4ba7d3a36efe2e5f3eff50e8c54"),
                    390,
                    5,
                    1,
                    bytes.fromhex("3045022100fae9ab97090f2f0fb5715497e12a06438cbccc610bae2f9c019dfa5bdb40f0090220283f5498f22e17ac9ecf4c239d864811dd47cb0ccb8c3584794791fd171e6b90"),
                    bytes.fromhex("0cfd04ed0b2b279c12412687c770b1224c8bfed453292652694339ddade4d63a")),
            ],
            1626626571,
            100000,
            bytes.fromhex('000071f1c701e06e5b91adb4289d6c5227b614bd4441748923826e5d0e8828da'),
            83651)

In [42]:
# get the byte array to use for computing the hash
first_block = get_test_first_block()
first_block_bytes = first_block.to_bytes()

# create a numpy array from the bytes
first_block_data = np.frombuffer(first_block_bytes, dtype=np.uint32)
print(first_block_data.size)
print(first_block_data)

19
[         0          0          0          0          0          0
          0          0 2057453135 3194810490 1327337811  812069561
 1512388052 1626626569          0     100000          0          0
          0]


In [36]:
# get the byte array to use for computing the hash for the second block
second_block = get_transactions_block()
second_block_bytes = second_block.to_bytes()

# create a numpy array from the bytes
second_block_data = np.frombuffer(second_block_bytes, dtype=np.uint32)
print(second_block_data)

[1058603008 3990394739 3650177306  781028127 2438392661 1129154654
 4038846301  905151409 2057453135 3194810490 1327337811  812069561
 1512388052 3976527116 2619812619 2267431186  582054087 3573451596
 1378232659 3711517545  987161773 1626626571          0     100000
          0          0          0]
